In [30]:
from scipy.optimize import minimize
import numpy as np

In [143]:
# error function parameters  modes
eta =0.01638284
'''rho = pm.Uniform("rho", lower=0, upper=10)
beta_L = pm.Uniform("beta_L", lower=0, upper=3)
beta_H = pm.Uniform("beta_H", lower=0, upper=6)'''
rho = 840.51783809
beta_L = 1.46443914 
beta_H = 1.13385676	
# rho_2 = pm.Uniform("rho_2", lower=0, upper=1)
beta_H2 = 0.19995312 
# alpha_1 = pm.Normal("alpha_1", mu=10, sigma=5)
alpha_L = 20.64151615 
alpha_1 = 11.60948562
alpha_2 = -2.64125169

sigma_lg_Phi =0.07716418   # standard deviation of the log of the error function

# computational cost
C_L = (0.02092 + 0.04517)*56 # CPU hours # 1.97  # C_L1 + C_L2 in CPU hours
C_H = 4.348*56  # 230.59

q = C_H / C_L

# objective error
err_0 = .0105 # goal error


In [144]:
q

65.78907550310181

In [146]:
def err_func(n_LF, n_HF):
    Phi = eta * (rho / ( (n_LF + n_HF) * (n_LF + alpha_L)**(beta_L-1) * (n_HF + alpha_1)**(beta_H-1) ) + 1 / (alpha_2 + n_HF)**(beta_H2))
    # median_Phi = 10**lg_Phi
    return Phi

def err_mean(n_LF, n_HF):
    gamma = np.exp(1/2 * (sigma_lg_Phi * np.log(10))**2)
    return gamma * err_func(n_LF, n_HF)

def comp_bud(n_L, n_H):
    return C_L * n_L + C_H * n_H

In [147]:


# # Define the objective function
def objective(vars):
    n_L, n_H = vars
    return comp_bud(n_L, n_H)

# Define the constraint function
def constraint(vars):
    n_L, n_H = vars
    return err_mean(n_L,n_H) - err_0

# Initial guess for the variables
initial_guess = [400, 10]

# Define the constraint dictionary
con = {'type': 'eq', 'fun': constraint}

# Perform the optimization
result = minimize(objective, initial_guess, constraints=[con])

# Extract the optimal values of x and y
x_opt, y_opt = result.x

print(f"Optimal values: x = {x_opt}, y = {y_opt}")
print(f"Function value at optimal points: f(x, y) = {objective((x_opt, y_opt))}")

Optimal values: x = 557.8040078321183, y = 17.575405403256035
Function value at optimal points: f(x, y) = 6343.85525597499


In [148]:
comp_bud(x_opt, y_opt) / 56 * 1200 

135939.75548517835

In [149]:
comp_bud(564, 19) / 56 * 1200 

143864.112

In [150]:
comp_bud(564, 15) / 56 * 1200 

122993.71200000001

In [151]:
comp_bud(564, 0) / 56 * 1200 

44729.71200000001

In [152]:
comp_bud(564, 6) / 56 * 1200 

76035.312

In [153]:
comp_bud(0, 1) / 56 * 1200 

5217.599999999999

In [154]:
err_mean(564,18)

0.010431166949162794

In [155]:
err_mean(564,15)

0.010874824811156973

In [180]:
n_LF = 564
n_HF = 15

In [181]:
err_mean(n_LF,n_HF)

0.010874824811156973

In [158]:
err_mean(564,6)

0.013929904407246495

In [159]:
comp_bud(n_LF, n_HF) / 56 * 1200

122993.71200000001

In [184]:
err_mean(0, 290)

0.010872920142444191

In [186]:
C_singlef = comp_bud(0, 290) / 56 * 1200

In [199]:
C_mfbox = comp_bud(564, 15) / 56 * 1200
print('Single-fidelity is ', C_singlef / C_mfbox, ' times more expensive than MF-box')
print("In another word, MF-Box reduces the computational cost by %f" % (100*(1 - C_mfbox/C_singlef)) + "%")


Single-fidelity is  12.302287453524452  times more expensive than MF-box
In another word, MF-Box reduces the computational cost by 91.871430%


In [162]:
err_func(10000, 0)

(0.010929435769951755-0.007928255586621727j)

In [163]:
# error function at integer n_LF and n_HF
n_LF = 78
n_HF = 9
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 78, n_HF = 9: 2.47%
C at n_LF = 78, n_HF = 9: 2480.07312 CPU hours = 44.29 node hours


In [164]:
(159*(564*2 + 15*64) * 9/8 + 2 * 8.4 * (564 + 564 * 4 + 15 *4**3)) / 1024

426.7529296875

In [165]:
def storage(n_LF, n_HF): # in TB
    return (159*(n_LF*2 + n_HF*64) * 9/8 + 2 * 8.4 * (n_LF + n_LF * 4 + n_HF *4**3))/1024

In [166]:
storage(564, 9)  # naive error function = 1.04%, C = 91688

353.3748046875

In [167]:
122993 - 76000

46993

In [168]:
storage(564, 6) 

316.6857421875

In [169]:
storage(564, 15) # new error function = 1.08%, C = 122993

426.7529296875

In [170]:
storage(564, 18) # new error function = 1.02%, C = 154299

463.4419921875